In [3]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter
data_folder_path = 'C:\\Users\\goodluck\\Desktop\\DB'
raw_data_path =  "C:\\Users\\goodluck\\Desktop\\DB\\raw_data"
storage_path =  "C:\\Users\\goodluck\\Desktop\\DB\\database_storage"
daily_path =  "C:\\Users\\goodluck\\Desktop\\DB\\daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")

In [4]:
# re-open
with open(DCBS_path, 'rb') as f:  
    DCBS = pickle.load(f)
# re-open
with open(DTBS_path, 'rb') as f:  
    DTBS = pickle.load(f)

In [27]:
def select_condition(codes, dt, fcd):
    satisfy = []
    if fcd[0] == 'lp':
        lm = calculate_time(dt, 'p', 20)
        
        for code in codes:
            if DCBS[lm[0]].__contains__(code):
                if low_prc(code, dt, lm, fcd[1]):
                    satisfy.append(code)
        
    else:
        for code in codes:
            if DCBS[dt][code][fcd[0]] <= fcd[2] and  DCBS[dt][code][fcd[0]] >= fcd[1]:
                satisfy.append(code)
                    
    return satisfy


def takeSecond(elem):
    return elem[1]

def rank(codes, dt, idx, num, d):
    rk = []
    if num == -1:
        num = len(codes)
    for code in codes:
        if idx == 'lp':
            if not np.isnan(DCBS[dt][code]['dp']):
                lm = calculate_time(dt, 'p', 20)
                if DCBS[lm[0]].__contains__(code):
                    prc = get_prcs(code, lm)
                    lp = absolute_low(prc, 0)
                    rk.append((code,  DCBS[dt][code]['dp']/lp))
            
            
        elif idx != 'dl':
            if not np.isnan( DCBS[dt][code][idx]):
                rk.append((code,  DCBS[dt][code][idx]))
        else:
            if not np.isnan( DCBS[dt][code][idx]):
                if  DCBS[dt][code][idx] != 999:
                    rk.append((code,  DCBS[dt][code][idx]))
    if num >= len(rk):
        num = len(rk)
        
     
#     reverse = True 降序
    if d == 'd':
        rk.sort(key=takeSecond, reverse = True)
    else:
        rk.sort(key=takeSecond, reverse = False)

    if idx == 'lp':
        lp = [i for i in rk if i[1] == 1.0]
        ot = []
        if num - len(lp) > 0:
            ot = rk[len(lp):num]
        
        low = lp + ot
        
    else:    
        low = rk[0:num]
#     print("In",dt , len(low), "of", len(codes),  "is selected")   

#     print(low)
    
    
    return [i[0] for i in low]

def get_prcs(code, dts):
    prc = []
    
    for dt in dts:
        prc.append(DCBS[dt][code]['dp'])
    prc.sort()
    return prc
    
def relative_low(lst, pos):
    return np.quantile(a=lst, q = pos)
    
def absolute_low(lst, pos):
    return min(lst) + pos * (max(lst) - min(lst))

def low_prc(code, dt, lm, pos):
    prc = get_prcs(code, lm)
#     lp = min([relative_low(prc, pos), absolute_low(prc, pos)])
    lp = absolute_low(prc, pos)
    tp = DCBS[dt][code]['dp']
#     if (tp <= lp):
#         print(code, tp, lp)
#     if tp <= lp:
#         print(tp, lp)
    return tp <= lp

def calculate_time(dt, d, l):
    time = []
    days = list(DCBS.keys())
    if d == 'p':
        for i in range(len(days)):
            if days[i] == dt:
                break
                
    return days[i-l+1:i+1]



In [14]:
tdd =datetime.today()
td = tdd.strftime('%Y-%m-%d')
td
DCBS.keys()
DCBS['2023-05-29']

{'110084.SH': {'cpr': -5.26,
  'dp': 133.4,
  'dl': 128.14000000000001,
  'ytm': -3.28,
  'bl': 9.994,
  'trt': 79.25,
  'yl': 4.584,
  'csp': 7.18,
  'ia': 1,
  'ipo': '2022-01-18',
  'cn': '贵燃转债',
  'sc': '600903.SH',
  'sn': '贵州燃气',
  'cat1': nan,
  'cat2': nan,
  'cl': 10.11,
  'pb': 3.74},
 '110057.SH': {'cpr': -0.27,
  'dp': 127.094,
  'dl': 126.824,
  'ytm': nan,
  'bl': 4.4,
  'trt': 11.13,
  'yl': nan,
  'csp': 9.51,
  'ia': 1,
  'ipo': '2019-04-30',
  'cn': '现代转债',
  'sc': '600420.SH',
  'sn': '国药现代',
  'cat1': nan,
  'cat2': nan,
  'cl': 12.12,
  'pb': 1.48},
 '113585.SH': {'cpr': -0.26,
  'dp': 213.266,
  'dl': 213.006,
  'ytm': nan,
  'bl': 0.344,
  'trt': 264.97,
  'yl': nan,
  'csp': 21.64,
  'ia': 1,
  'ipo': '2020-07-07',
  'cn': '寿仙转债',
  'sc': '603896.SH',
  'sn': '寿仙谷',
  'cat1': nan,
  'cat2': nan,
  'cl': 46.27,
  'pb': 4.66},
 '127058.SZ': {'cpr': 0.01,
  'dp': 203.451,
  'dl': 203.46099999999998,
  'ytm': -11.77,
  'bl': 20.62,
  'trt': 5.79,
  'yl': 4.808,
  'c

In [28]:
select = list(DCBS[td].keys())

flt_cd = [['dp', 100, 130]]

flt_cd = [['dp', 100, 130], ['bl',0, 3]]

flt_cd = [['dp', 100, 130], ['bl',0, 3]]

flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]



flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', 0.5, 9999]]
flt_cd = [['dp', 100, 130], ['bl',0, 3]]

flt_cd = [['dp', 100, 130], ['bl',0, 3]]

flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', 0.5, 9999]]
flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]

flt_cd = [['lp', 0.5], ['dp', 100, 130], ['bl',0, 3], ['cpr', -999, 50]]
for fcd in flt_cd:
    select = select_condition(select, td, fcd)

print(len(select))
    
# rk_cd = [['lp', 10, 'a']]
rk_cd = [['dl', 600, 'a']]



rk_cd = [['dl', 30, 'a']]



rk_cd = [['dl', 30, 'a']]


rk_cd = [['ytm', 20, 'd']]
rk_cd = [['dl', 30, 'a']]
rk_cd = [['lp', 30, 'a']]
codes = rank(select, td, rk_cd[0][0], rk_cd[0][1], rk_cd[0][2])
len(codes)

17


17

In [29]:
for r in codes:
    print(r, DCBS[td][r]['cn'], DCBS[td][r]['sc'],  DCBS[td][r]['sn'], "收盘价：", DCBS[td][r]['dp'],  "转股溢价率：", DCBS[td][r]['cpr'], "到期收益率：", DCBS[td][r]['ytm'],"换手率：", DCBS[td][r]['trt'], '市值',  DCBS[td][r]['bl'], '双低', DCBS[td][r]['dl'])

113595.SH 花王转债 603007.SH ST花王 收盘价： 104.986 转股溢价率： 18.17 到期收益率： 5.57 换手率： 17.39 市值 2.707 双低 123.156
128039.SZ 三力转债 002224.SZ 三力士 收盘价： 113.993 转股溢价率： 45.85 到期收益率： -5.62 换手率： 0.98 市值 2.024 双低 159.843
128021.SZ 兄弟转债 002562.SZ 兄弟科技 收盘价： 110.016 转股溢价率： 32.39 到期收益率： -7.29 换手率： 1.56 市值 2.651 双低 142.406
113566.SH 翔港转债 603499.SH 翔港科技 收盘价： 119.104 转股溢价率： 40.5 到期收益率： 0.89 换手率： 2.07 市值 1.613 双低 159.60399999999998
128053.SZ 尚荣转债 002551.SZ 尚荣医疗 收盘价： 124.976 转股溢价率： 47.31 到期收益率： -6.34 换手率： 4.82 市值 1.898 双低 172.286
110070.SH 凌钢转债 600231.SH 凌钢股份 收盘价： 115.978 转股溢价率： 36.54 到期收益率： -0.12 换手率： 1.62 市值 2.171 双低 152.518
113561.SH 正裕转债 603089.SH 正裕工业 收盘价： 118.234 转股溢价率： 40.64 到期收益率： -0.75 换手率： 2.61 市值 1.499 双低 158.874
113526.SH 联泰转债 603797.SH 联泰环保 收盘价： 125.403 转股溢价率： 23.89 到期收益率： -8.82 换手率： 3.02 市值 1.927 双低 149.293
113030.SH 东风转债 601515.SH 东风股份 收盘价： 119.135 转股溢价率： 34.0 到期收益率： -1.28 换手率： 0.9 市值 2.945 双低 153.135
128056.SZ 今飞转债 002863.SZ 今飞凯达 收盘价： 127.112 转股溢价率： 36.93 到期收益率： -4.57 换手率： 6.13 市值 1.72 双低 164.042
12300

In [41]:
tdd =datetime.today()
td = tdd.strftime('%Y-%m-%d')
select = list(DCBS[td].keys())
for code in select:
    DCBS[td][code]['cat1'] = DTBS['B'][code]['cat1']
    DCBS[td][code]['cat2'] = DTBS['B'][code]['cat2']
    
# save
f_save = open(DCBS_path, 'wb')
pickle.dump(DCBS, f_save)
f_save.close()


In [34]:
DTBS['127082.SZ']

KeyError: '127082.SZ'

In [9]:
cbcodes = []
stcodes = []

cbcodes = list(DTBS['A'].keys())

for cb in cbcodes:
    st = DTBS['B'][cb]['sc']
    if st not in stcodes:
        stcodes.append(st)

In [11]:
len(stcodes)

722

In [13]:
df = pd.DataFrame(cbcodes)

In [15]:
df2 = pd.DataFrame(stcodes)

In [16]:
df.to_csv()

',0\r\n0,110088.SH\r\n1,113545.SH\r\n2,113631.SH\r\n3,123128.SZ\r\n4,123156.SZ\r\n5,127027.SZ\r\n6,127061.SZ\r\n7,110047.SH\r\n8,110063.SH\r\n9,110070.SH\r\n10,110087.SH\r\n11,110089.SH\r\n12,110092.SH\r\n13,111001.SH\r\n14,111002.SH\r\n15,111003.SH\r\n16,111007.SH\r\n17,111012.SH\r\n18,113025.SH\r\n19,113027.SH\r\n20,113046.SH\r\n21,113047.SH\r\n22,113058.SH\r\n23,113059.SH\r\n24,113064.SH\r\n25,113534.SH\r\n26,113535.SH\r\n27,113566.SH\r\n28,113591.SH\r\n29,113600.SH\r\n30,113615.SH\r\n31,113621.SH\r\n32,113623.SH\r\n33,113632.SH\r\n34,113636.SH\r\n35,113638.SH\r\n36,113640.SH\r\n37,113641.SH\r\n38,113649.SH\r\n39,113650.SH\r\n40,113653.SH\r\n41,113657.SH\r\n42,113663.SH\r\n43,118000.SH\r\n44,118005.SH\r\n45,118006.SH\r\n46,118008.SH\r\n47,118010.SH\r\n48,118018.SH\r\n49,118020.SH\r\n50,118022.SH\r\n51,118023.SH\r\n52,118028.SH\r\n53,118029.SH\r\n54,123012.SZ\r\n55,123013.SZ\r\n56,123031.SZ\r\n57,123046.SZ\r\n58,123052.SZ\r\n59,123057.SZ\r\n60,123076.SZ\r\n61,123078.SZ\r\n62,123085.S

In [19]:
df.to_csv('cb.csv')
df2.to_csv('st.csv')